In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pennylane import numpy as np
from sklearn.decomposition import PCA
import single_model_train
import qcnn_estimator
import data

In [2]:
samples = data.data_load_and_split(["rock", "country"])

estimator = qcnn_estimator.QCNNEstimator()
pipeline = Pipeline(steps=[
  ("scaler", MinMaxScaler((0, np.pi / 2))),
  ("pca", PCA(n_components=8)),
  ("model", estimator)
])

In [3]:
#--- Grid Search for hyperparameters
grid_params = { 'model__stride_c':list(range(2,3)),
                'model__filter_p':["!*"]
              }

grid = GridSearchCV(pipeline, grid_params, cv=5, n_jobs=8, verbose=True, refit=True)

In [4]:
grid.fit(samples.x_train, samples.y_train.values)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


Loss at step 0: 0.670102069081626
Loss at step 0: 0.7098038346722519
Loss at step 0: 0.6108208934605528
Loss at step 0: 0.6288807419905071
Loss at step 0: 0.6831737744748165
Loss at step 25: 0.5934791726974582
Loss at step 25: 0.5667366786696789
Loss at step 25: 0.5770803501843519
Loss at step 25: 0.5787217279068285
Loss at step 25: 0.5904535143960434
Loss at step 50: 0.5644424337820296
Loss at step 50: 0.5900438054092404
Loss at step 50: 0.5760717760352357
Loss at step 50: 0.5774751801624295
Loss at step 50: 0.5896134384226103
Loss at step 75: 0.5899550313027946
Loss at step 75: 0.5641370578921323
Loss at step 75: 0.5760374074355752
Loss at step 75: 0.5768739710878024
Loss at step 75: 0.5895484726643028
Loss at step 0: 0.6971483904056888
Loss at step 25: 0.5925655445012787
Loss at step 50: 0.5868547729379664
Loss at step 75: 0.5865780347418672


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler',
                                        MinMaxScaler(feature_range=(0,
                                                                    1.5707963267948966))),
                                       ('pca', PCA(n_components=8)),
                                       ('model', QCNNEstimator(stride_c=2))]),
             n_jobs=8,
             param_grid={'model__filter_p': ['!*'], 'model__stride_c': [2]},
             verbose=True)

In [5]:
# Get the best model
best_circuit = grid.best_estimator_
# Print the best hyperparameters
print('=========================================[Best Hyperparameters info]=====================================')
# summarize best
print('Best score: %.3f'  % grid.best_score_)
print('Best Config: %s' % grid.best_params_)
print('==========================================================================================================')

=========================================[Best Hyperparameters info]=====================================
Best score: 0.686
Best Config: {'model__filter_p': '!*', 'model__stride_c': 2}


In [6]:
best_circuit.score(samples.x_test, samples.y_test.values)

0.6666666666666666

In [7]:
import pandas as pd
results = pd.DataFrame(grid.cv_results_)

In [8]:
import os
os.makedirs('results/', exist_ok=True)

In [9]:
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
print(f"results/out_{date}.csv")

results/out_2024-03-25_14-11-18.csv


In [10]:
results.to_csv(f"results/out_{date}.csv", index=False)

In [20]:
best_circuit.named_steps.model.symbols

tensor([ 0.9329,  1.3964, -0.3752, -0.1341, -0.4830,  0.5225],
       requires_grad=True)